# 01 — Data Ingestion & Overview
Notes:
- Objective: locally stage raw datasets, inspect columns and sample rows, basic cleaning utilities.
- Outputs: processed master table saved to data/processed/master_table.parquet


In [4]:
# Notebook 01: Data Loading & Initial Cleaning
# T-IQ Project

import pandas as pd
from pathlib import Path
import json
import re

# ------------------------------
# Setup paths
# ------------------------------
RAW = Path(r"C:\Users\abanu\Documents\T-IQ\data\raw")
PROCESSED = Path(RAW.parent / "processed")
PROCESSED.mkdir(exist_ok=True, parents=True)

print("Raw data path:", RAW)
print("Processed data path:", PROCESSED)


Raw data path: C:\Users\abanu\Documents\T-IQ\data\raw
Processed data path: C:\Users\abanu\Documents\T-IQ\data\processed


In [5]:
# ------------------------------
# 1. Load IBM HR Attrition
# ------------------------------
attrition_path = RAW / "WA_Fn-UseC_-HR-Employee-Attrition.csv"
attrition = pd.read_csv(attrition_path)
print("Attrition dataset shape:", attrition.shape)
attrition.head()


Attrition dataset shape: (1470, 35)


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [6]:
# Basic cleaning example
attrition = attrition.drop_duplicates()
attrition['Attrition'] = attrition['Attrition'].map({'Yes':1, 'No':0})

# Save processed
attrition.to_csv(PROCESSED / "attrition_clean.csv", index=False)
print("Attrition cleaned and saved.")


Attrition cleaned and saved.


In [7]:
# ------------------------------
# 2. Load HRMS Summary
# ------------------------------
hrms_summary_path = RAW / "hrms_synth_summary.csv"
hrms_summary = pd.read_csv(hrms_summary_path)
print("HRMS summary shape:", hrms_summary.shape)
hrms_summary.head()


HRMS summary shape: (10000, 11)


,employee_id,name,department,job_role,location,current_salary,satisfaction_score,engagement_score,num_skills,years_at_company,trainings_count
0,EMP000001,Vikram Singh,HR,Data Scientist,"New York, USA",4544478,0.78,0.80,7,12,0
1,EMP000002,Karan Patel,Marketing,Data Scientist,"Chennai, India",5180268,0.71,0.93,8,7,4
2,EMP000003,Vikram Malhotra,Marketing,Senior Software Engineer,"Chennai, India",2589268,0.81,0.56,6,3,3
3,EMP000004,Siddharth Khan,HR,ML Engineer,"Bengaluru, India",1321856,0.43,0.95,7,15,3
4,EMP000005,Priya Nair,Legal,ML Engineer,Remote,4371479,0.41,0.70,4,7,2


In [8]:
# Remove duplicates
hrms_summary = hrms_summary.drop_duplicates()

# Save processed
hrms_summary.to_csv(PROCESSED / "hrms_summary_clean.csv", index=False)
print("HRMS summary cleaned and saved.")


HRMS summary cleaned and saved.


In [9]:
# ------------------------------
# 3. Load Job Descriptions
# ------------------------------
jobs_path = RAW / "job_descriptions.csv"
jobs = pd.read_csv(jobs_path)
print("Job Descriptions shape:", jobs.shape)
jobs.head()


Job Descriptions shape: (10000, 12)


,job_id,job_title,company,location,industry,seniority,employment_type,salary_min,salary_max,years_experience,skills,job_text
0,JOB000001,Cloud Architect,TechNova29,"Bengaluru, India",EdTech,Mid,Part-time,107397,140048,0,Quantitative Analysis;Azure;Feature Engineerin...,"Design, develop and maintain scalable Kubernet..."
1,JOB000002,HR Specialist,TechNova2,"Dublin, Ireland",Healthcare,Lead,Contract,42676,71202,5,Spark;Node.js;spaCy;Azure;Kubernetes;GANs,Maintain documentation and operate within Agil...
2,JOB000003,Systems Analyst,TechNova59,"Dublin, Ireland",EdTech,Entry,Part-time,117841,140337,10,AWS;GANs;Kotlin;RESTful Design;Tableau;RoBERTa...,Collaborate with cross-functional teams to def...
3,JOB000004,Data Engineer,TechNova146,Remote,FinTech,Mid,Internship,103579,143901,4,Looker;RESTful Design;Git;Jira;Linux;CI/CD;Fin...,Maintain documentation and operate within Agil...
4,JOB000005,Data Scientist,TechNova99,"San Francisco, USA",Manufacturing,Lead,Remote,128419,150905,10,Leadership;Java;Hyperparameter Tuning;Customer...,Develop proof-of-concepts for new ML models an...


In [10]:
# Clean text column
if 'job_text' in jobs.columns:
    jobs['job_text'] = jobs['job_text'].str.lower().str.replace(r'\s+', ' ', regex=True)
    jobs['job_text'] = jobs['job_text'].str.replace(r'[^a-z0-9 ,.]', '', regex=True)

# Save processed
jobs.to_csv(PROCESSED / "job_descriptions_clean.csv", index=False)
print("Job descriptions cleaned and saved.")


Job descriptions cleaned and saved.


In [11]:
# ------------------------------
# 4. Load Glassdoor Reviews
# ------------------------------
reviews_path = RAW / "glassdoor_reviews.csv"
reviews = pd.read_csv(reviews_path)
print("Reviews shape:", reviews.shape)
reviews.head()


Reviews shape: (10000, 10)


,review_id,company,job_title,location,rating,pros,cons,review_text,recommend,date
0,REV000001,TechNova47,Backend Developer,"Toronto, Canada",5,Supportive management and mentorship programs.,Onboarding could be improved for new hires.,"As a Backend Developer at TechNova47, I enjoye...",No,2023-11-16
1,REV000002,TechNova194,QA Engineer,"Pune, India",3,Supportive management and mentorship programs.,Tight deadlines during peak releases.,"As a QA Engineer at TechNova194, I enjoyed Sup...",No,2021-09-19
2,REV000003,TechNova45,Network Engineer,"Berlin, Germany",2,Great team and collaborative environment.,Tight deadlines during peak releases.,The company culture is toxic. My main positive...,Maybe,2022-01-21
3,REV000004,TechNova15,Cloud Architect,Singapore,4,Open culture and emphasis on innovation.,Limited documentation for legacy systems.,"As a Cloud Architect at TechNova15, I enjoyed ...",No,2025-09-20
4,REV000005,TechNova191,Senior Software Engineer,"Paris, France",1,Open culture and emphasis on innovation.,Onboarding could be improved for new hires.,The company culture is supportive. My main pos...,Yes,2022-06-21


In [12]:
# Clean text columns
for col in ['review_text', 'pros', 'cons']:
    if col in reviews.columns:
        reviews[col] = reviews[col].astype(str).str.lower().str.replace(r'\s+', ' ', regex=True)
        reviews[col] = reviews[col].str.replace(r'[^a-z0-9 ,.]', '', regex=True)

# Save processed
reviews.to_csv(PROCESSED / "glassdoor_reviews_clean.csv", index=False)
print("Glassdoor reviews cleaned and saved.")


Glassdoor reviews cleaned and saved.


In [13]:
# ------------------------------
# 5. Load Resume Metadata
# ------------------------------
resumes_path = RAW / "resumes/Resume/Resume.csv"
resumes = pd.read_csv(resumes_path)
print("Resumes metadata shape:", resumes.shape)
resumes.head()


Resumes metadata shape: (2484, 4)


,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [14]:
# Clean skills column (if exists)
if 'skills' in resumes.columns:
    resumes['skills'] = resumes['skills'].astype(str).str.lower().str.replace(r'\s+', ' ', regex=True)
    resumes['skills'] = resumes['skills'].str.replace(r'[^a-z0-9 ,]', '', regex=True)

# Save processed
resumes.to_csv(PROCESSED / "resumes_clean.csv", index=False)
print("Resumes metadata cleaned and saved.")


Resumes metadata cleaned and saved.


In [15]:
# ------------------------------
# 6. Load HRMS Full JSON
# ------------------------------
hrms_json_path = RAW / "hrms_synth.json"
with open(hrms_json_path, "r") as f:
    hrms_json = json.load(f)

print("Number of HRMS JSON records:", len(hrms_json))


Number of HRMS JSON records: 10000


In [16]:
# Flatten JSON to DataFrame
hrms_flat = pd.json_normalize(hrms_json)
print("HRMS JSON flattened shape:", hrms_flat.shape)
hrms_flat.head()

# Save processed
hrms_flat.to_csv(PROCESSED / "hrms_json_flattened.csv", index=False)
print("HRMS JSON flattened and saved.")


HRMS JSON flattened shape: (10000, 26)
HRMS JSON flattened and saved.


In [17]:
print("All datasets loaded, cleaned, and saved in:", PROCESSED)


All datasets loaded, cleaned, and saved in: C:\Users\abanu\Documents\T-IQ\data\processed
